# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 31 2022 9:44AM,238383,10,0085720732,ISDIN Corporation,Released
1,May 31 2022 9:44AM,238383,10,0085720789,ISDIN Corporation,Released
2,May 31 2022 9:44AM,238383,10,0085720804,ISDIN Corporation,Released
3,May 31 2022 9:44AM,238383,10,0085720809,ISDIN Corporation,Released
4,May 31 2022 9:44AM,238383,10,0085720810,ISDIN Corporation,Released
5,May 31 2022 9:44AM,238383,10,0085720816,ISDIN Corporation,Released
6,May 31 2022 9:44AM,238383,10,0085720819,ISDIN Corporation,Released
7,May 31 2022 9:44AM,238383,10,0085720825,ISDIN Corporation,Released
8,May 31 2022 9:44AM,238383,10,0085720818,ISDIN Corporation,Released
9,May 31 2022 9:44AM,238383,10,0085720826,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,238381,Released,60
34,238382,Released,1
35,238383,Released,51
36,238384,Released,1
37,238385,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238381,NaN,NaN,60.0
238382,NaN,NaN,1.0
238383,NaN,NaN,51.0
238384,NaN,NaN,1.0
238385,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238334,0.0,0.0,14.0
238335,0.0,1.0,0.0
238339,0.0,0.0,1.0
238341,0.0,0.0,35.0
238344,1.0,3.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238334,0,0,14
238335,0,1,0
238339,0,0,1
238341,0,0,35
238344,1,3,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238334,0,0,14
1,238335,0,1,0
2,238339,0,0,1
3,238341,0,0,35
4,238344,1,3,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238334,,,14
1,238335,,1,
2,238339,,,1
3,238341,,,35
4,238344,1,3,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 31 2022 9:44AM,238383,10,ISDIN Corporation
51,May 31 2022 9:42AM,238385,12,"Uniderm USA, Inc"
53,May 31 2022 9:41AM,238384,10,"Citieffe, Inc."
54,May 31 2022 9:38AM,238382,18,"ClearSpec, LLC"
55,May 31 2022 9:37AM,238381,10,ISDIN Corporation
115,May 31 2022 9:32AM,238376,10,ISDIN Corporation
162,May 31 2022 9:32AM,238380,10,Beach Products Inc
163,May 31 2022 9:31AM,238378,10,"Amcyte Pharma, Inc."
168,May 31 2022 9:31AM,238377,10,"Citieffe, Inc."
169,May 31 2022 9:28AM,238374,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 31 2022 9:44AM,238383,10,ISDIN Corporation,,,51
1,May 31 2022 9:42AM,238385,12,"Uniderm USA, Inc",,,2
2,May 31 2022 9:41AM,238384,10,"Citieffe, Inc.",,,1
3,May 31 2022 9:38AM,238382,18,"ClearSpec, LLC",,,1
4,May 31 2022 9:37AM,238381,10,ISDIN Corporation,,,60
5,May 31 2022 9:32AM,238376,10,ISDIN Corporation,,,47
6,May 31 2022 9:32AM,238380,10,Beach Products Inc,,,1
7,May 31 2022 9:31AM,238378,10,"Amcyte Pharma, Inc.",,,5
8,May 31 2022 9:31AM,238377,10,"Citieffe, Inc.",,,1
9,May 31 2022 9:28AM,238374,10,ISDIN Corporation,,,48


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 9:44AM,238383,10,ISDIN Corporation,51,,
1,May 31 2022 9:42AM,238385,12,"Uniderm USA, Inc",2,,
2,May 31 2022 9:41AM,238384,10,"Citieffe, Inc.",1,,
3,May 31 2022 9:38AM,238382,18,"ClearSpec, LLC",1,,
4,May 31 2022 9:37AM,238381,10,ISDIN Corporation,60,,
5,May 31 2022 9:32AM,238376,10,ISDIN Corporation,47,,
6,May 31 2022 9:32AM,238380,10,Beach Products Inc,1,,
7,May 31 2022 9:31AM,238378,10,"Amcyte Pharma, Inc.",5,,
8,May 31 2022 9:31AM,238377,10,"Citieffe, Inc.",1,,
9,May 31 2022 9:28AM,238374,10,ISDIN Corporation,48,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 9:44AM,238383,10,ISDIN Corporation,51,,
1,May 31 2022 9:42AM,238385,12,"Uniderm USA, Inc",2,,
2,May 31 2022 9:41AM,238384,10,"Citieffe, Inc.",1,,
3,May 31 2022 9:38AM,238382,18,"ClearSpec, LLC",1,,
4,May 31 2022 9:37AM,238381,10,ISDIN Corporation,60,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 9:44AM,238383,10,ISDIN Corporation,51.0,NaN,NaN
1,May 31 2022 9:42AM,238385,12,"Uniderm USA, Inc",2.0,NaN,NaN
2,May 31 2022 9:41AM,238384,10,"Citieffe, Inc.",1.0,NaN,NaN
3,May 31 2022 9:38AM,238382,18,"ClearSpec, LLC",1.0,NaN,NaN
4,May 31 2022 9:37AM,238381,10,ISDIN Corporation,60.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 31 2022 9:44AM,238383,10,ISDIN Corporation,51.0,0.0,0.0
1,May 31 2022 9:42AM,238385,12,"Uniderm USA, Inc",2.0,0.0,0.0
2,May 31 2022 9:41AM,238384,10,"Citieffe, Inc.",1.0,0.0,0.0
3,May 31 2022 9:38AM,238382,18,"ClearSpec, LLC",1.0,0.0,0.0
4,May 31 2022 9:37AM,238381,10,ISDIN Corporation,60.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4529000,487.0,11.0,0.0
102,238367,1.0,0.0,0.0
12,1430121,80.0,0.0,0.0
18,476757,2.0,0.0,0.0
19,476698,8.0,3.0,1.0
20,476669,14.0,1.0,0.0
22,476730,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4529000,487.0,11.0,0.0
1,102,238367,1.0,0.0,0.0
2,12,1430121,80.0,0.0,0.0
3,18,476757,2.0,0.0,0.0
4,19,476698,8.0,3.0,1.0
5,20,476669,14.0,1.0,0.0
6,22,476730,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,487.0,11.0,0.0
1,102,1.0,0.0,0.0
2,12,80.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,8.0,3.0,1.0
5,20,14.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,487.0
1,102,Released,1.0
2,12,Released,80.0
3,18,Released,2.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,18,19,20,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,11.0,0.0,0.0,0.0,3.0,1.0,0.0
Released,487.0,1.0,80.0,2.0,8.0,14.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,11.0,0.0,0.0,0.0,3.0,1.0,0.0
2,Released,487.0,1.0,80.0,2.0,8.0,14.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,18,19,20,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,11.0,0.0,0.0,0.0,3.0,1.0,0.0
2,Released,487.0,1.0,80.0,2.0,8.0,14.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()